In [ ]:
# This is my attempt to initialise BERT within fastai. 
#It is based on the tutorial by Keita Kurita: https://github.com/keitakurita/Practical_NLP_in_PyTorch/blob/master/fastai/bert_with_fastai.ipynb

In [30]:
import numpy as np
import pandas as pd

from pathlib import Path
from typing import *

import torch
import torch.optim as optim

In [39]:
from fastai import *
from fastai.vision import *
from fastai.text.all import *
from transformers import BertTokenizer

In [33]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)

config = Config(
    testing=False,
    bert_model_name="bert-base-uncased",
    max_lr=3e-5,
    epochs=1,
    use_fp16=False,
    bs=4,
    discriminative=False,
    max_seq_len=128,
)

In [34]:
bert_tok = BertTokenizer.from_pretrained(
    config.bert_model_name,
)


In [35]:
class FastAiBertTokenizer(BaseTokenizer):
    """Wrapper around BertTokenizer to be compatible with fast.ai"""
    def __init__(self, tokenizer: BertTokenizer, max_seq_len: int=128, **kwargs):
        self._pretrained_tokenizer = tokenizer
        self.max_seq_len = max_seq_len

    def __call__(self, *args, **kwargs):
        return self

    def tokenizer(self, t:str) -> List[str]:
        """Limits the maximum sequence length"""
        return ["[CLS]"] + self._pretrained_tokenizer.tokenize(t)[:self.max_seq_len - 2] + ["[SEP]"]

In [37]:
fastai_bert_tokenizer = FastAiBertTokenizer(tokenizer=bert_tok, max_seq_len=config.max_seq_len)


In [40]:
fastai_bert_vocab = Vocab(list(bert_tok.vocab.keys()))
#The Vocab class is from fastai.text.all, which I have imported, so I'm not sure why it wont work here.
#I cant find anything suggesting that the class has been removed in an update.

NameError: name 'Vocab' is not defined